In [1]:
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
tf.enable_eager_execution()

In [2]:
from keras.preprocessing.image import load_img, img_to_array
image = tf.keras.preprocessing.image.load_img("image/content/mug.jpg", target_size=(224, 224))
image = tf.keras.preprocessing.image.img_to_array(image)
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
image = tf.keras.applications.vgg16.preprocess_input(image, mode='tf')
image = tf.convert_to_tensor(image)

Using TensorFlow backend.


In [3]:
image

<tf.Tensor: id=1, shape=(1, 224, 224, 3), dtype=float32, numpy=
array([[[[-0.4980392 , -0.5764706 , -0.9764706 ],
         [-0.4980392 , -0.5529412 , -0.9764706 ],
         [-0.38823527, -0.58431375, -1.        ],
         ...,
         [-0.08235294, -0.2862745 , -0.4980392 ],
         [-0.12941176, -0.3098039 , -0.654902  ],
         [-0.1607843 , -0.36470586, -0.4980392 ]],

        [[-0.42745095, -0.6156863 , -0.9607843 ],
         [-0.4823529 , -0.5529412 , -1.        ],
         [-0.5137255 , -0.5764706 , -0.9372549 ],
         ...,
         [-0.0745098 , -0.29411763, -0.58431375],
         [-0.04313725, -0.27843136, -0.56078434],
         [-0.25490195, -0.4980392 , -0.7254902 ]],

        [[-0.47450978, -0.56078434, -0.9843137 ],
         [-0.44313723, -0.5921569 , -0.92156863],
         [-0.44313723, -0.5686275 , -1.        ],
         ...,
         [-0.05882353, -0.29411763, -0.4823529 ],
         [-0.00392157, -0.2235294 , -0.5137255 ],
         [-0.18431371, -0.41960782, -0.6

In [4]:
# Pretrained VGG16 model on imagenet
model = tf.keras.applications.vgg16.VGG16()

In [5]:
pred = model.predict(image)
pred

array([[2.39911260e-05, 5.63646900e-04, 4.03788144e-05, 1.47099112e-04,
        3.24385735e-04, 2.73360347e-04, 6.57134340e-04, 7.16627037e-05,
        4.11621222e-05, 3.42438361e-05, 1.01884907e-04, 6.04970155e-05,
        1.44974911e-04, 1.98821363e-04, 7.54385692e-05, 3.83083607e-05,
        1.26059313e-04, 4.85496312e-05, 1.12044225e-04, 2.08921410e-04,
        7.22857803e-05, 6.03811895e-05, 6.48353380e-05, 3.94153139e-05,
        2.24149662e-05, 1.22052179e-05, 1.05008410e-04, 7.91886123e-05,
        1.18019498e-05, 1.94369070e-03, 2.30775622e-05, 8.52356170e-05,
        3.38731988e-05, 2.38862223e-04, 1.84203949e-04, 4.40958065e-05,
        1.99381975e-04, 3.93500741e-05, 2.91830132e-04, 6.09937451e-05,
        1.45297818e-04, 4.37292074e-05, 8.32537553e-05, 5.40238325e-05,
        6.13815573e-05, 6.25487650e-04, 1.30795670e-04, 1.74033121e-04,
        1.85218960e-04, 9.23207699e-05, 1.84218006e-04, 1.05557985e-04,
        3.01155262e-04, 3.26306821e-04, 1.49005486e-04, 1.071129

In [6]:
# convert the probabilities to class labels
label = tf.keras.applications.vgg16.decode_predictions(pred)
# retrieve the most likely result, e.g. highest probability
label = label[0][0]
# print the classification
print('%s (%.2f%%)' % (label[1], label[2]*100))

toilet_tissue (10.89%)


In [ ]:
model.summary()

In [ ]:
for layer in model.layers[:19]:
    print(layer.name)

In [ ]:
def style_features(model, image):
    """ Run an image forward through a model and get the features for 
        a set of style layers.
        Returns a dictionary of the layer name and the activations.
    """
    style_layers = ['block1_conv2', 'block2_conv2', 'block3_conv3', 'block4_conv3']
    
    features = {}
    x = image
    # model._modules is a dictionary holding each module in the model
    for layer in model.layers:
        x = layer(x)
        if layer.name in style_layers:
            features[layer.name] = x
            if layer.name == 'block4_conv3':
                break
            
    return features

def content_feature(model, image):
    """ Run an image forward through a model and get the features for 
        a set of conent layers.
        Returns the activation of the content layer
    """
    style_layers = ['block3_conv3']
    
    x = image
    # model._modules is a dictionary holding each module in the model
    for layer in model.layers:
        x = layer(x)
        if layer.name in style_layers:
            features = x
            break
            
    return features

In [ ]:
def gram_matrix(x):
    """ Compute gram matrix of a 3 dimensional convolution
    """
    b, h, w, c = tf.shape(x)
    x = tf.reshape(x, [b, c, -1])
    return tf.matmul(x, tf.transpose(x, perm=[0, 2, 1])) / (c * h * w)

In [ ]:
def loss(y, content_image, style_image, content_weight, style_weight):
    """ Compute loss of output with respect to content and style image
    """
    # Pretrained VGG16 on imagenet
    model = VGG16()
    
    # Style features of output
    output_style_features = style_features(model, output_image)
    # Content features of output
    output_content_feature = content_feature(model, output_image)
    
    # Style features of style image
    style_features = style_features(model, style_image)
    # Content features of content image
    content_feature = content_feature(model, content_image)
    
    # Compute content loss
    # (output - content )/(Cj * Hj * Wj)
    content_loss = content_weight * tf.reduce_sum(tf.reduce_mean(tf.math.square(output_content_feature - content_feature)))
    
    # Compute style loss
    # Gram matrix of output features
    output_gram = [gram_matrix(x) for _, x in output_style_features.items()]
    # Convert output gram to tensor for efficient computation
    output_gram = tf.convert_to_tensor(output_gram)
                                                  
    # Gram matrix of style features
    style_gram = [gram_matrix(x) for _, x in style_features.items()]
    # Convert style gram to tensor for efficient computation
    style_gram = tf.convert_to_tensor(style_gram)
                                                  
    style_loss = tf.square(tf.norm(output_gram - style_gram))
                                                  
    # TODO: Add total variation regularization
    
    total_loss = content_weight * content_loss + style_weight * style_loss
    return total_loss
    

In [ ]:
style_features(model, image)

In [ ]:
content_feature(model, image)